In [1]:
import pandas as pd
import numpy as np

In [2]:
argentina = pd.read_csv('../argentina.csv')
australia = pd.read_csv('../australia.csv')
france = pd.read_csv('../france.csv')
ireland = pd.read_csv('../ireland.csv')
new_zealand = pd.read_csv('../new-zealand.csv')
scotland = pd.read_csv('../scotland.csv')
south_africa = pd.read_csv('../south-africa.csv')
wales = pd.read_csv('../wales.csv')

frames = [argentina, australia, france, ireland, new_zealand, scotland, south_africa, wales]
combined = pd.concat(frames).reset_index(drop = True)

#Remove cm and kg from height and weight, calculate mass for each player
combined['Height'] = combined['Height'].map(lambda x: x.strip('cm')).astype(int)
combined['Weight'] = combined['Weight'].map(lambda x: x.strip('kg')).astype(int)
combined["Mass"] = (combined["Height"]/100) * combined["Weight"]

backs = ["Wing", "Centre", "Scrum Half", "Fly Half", "Full Back"]
forwards = ["Back Row", "Prop", "Hooker", "Lock"]

combined['Field_Pos'] = 0 #create a column for Field Position, assign all to 0
for position in np.unique(combined['Position']):
    if position in backs:
        combined.Field_Pos[combined.Position == position] = "Back" #If players postion is in the backs list assign Field_Pos to Back
    else:
        combined.Field_Pos[combined.Position == position] = "Forward" #Assign Field_Pos to Forward

#Create targets for the Linear Regressor to predict. The class to predict must be a number (0,1,2), not a string (Lock, Prop, Fly Half)
combined["Target"] = 0
combined.Target[combined.Field_Pos == "Forward"] = 1

combined.head()

C:\Users\maniacalbrain\Anaconda\lib\site-packages\IPython\kernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\maniacalbrain\Anaconda\lib\site-packages\IPython\kernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\maniacalbrain\Anaconda\lib\site-packages\IPython\kernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Name,Position,Debut,Age,Height,Weight,Matches,Points_Scored,Tries_Scored,Yellow_Cards,Red_Cards,Mass,Field_Pos,Target
0,Matias Alemanno,Lock,17 May 2014,23,198,112,18,10,2,0,0,221.76,Forward,1
1,Marcos Ayerza,Prop,04 Dec 2004,32,186,113,64,5,1,1,0,210.18,Forward,1
2,Agustín Creevy,Hooker,23 Apr 2005,30,182,106,45,10,2,0,0,192.92,Forward,1
3,Juan Pablo Orlandi,Prop,08 Nov 2008,32,190,119,19,0,0,0,0,226.10,Forward,1
4,Juan Martín Fernandez Lobbe,Back Row,28 Apr 2004,33,191,106,69,25,5,2,0,202.46,Forward,1


In [65]:
combined = combined.drop(["Position", "Debut", "Age"], axis = 1)
combined.head()

,Name,Height,Weight,Matches,Points_Scored,Tries_Scored,Yellow_Cards,Red_Cards,Mass,Field_Pos,Target
0,Matias Alemanno,198,112,18,10,2,0,0,221.76,Forward,1
1,Marcos Ayerza,186,113,64,5,1,1,0,210.18,Forward,1
2,Agustín Creevy,182,106,45,10,2,0,0,192.92,Forward,1
3,Juan Pablo Orlandi,190,119,19,0,0,0,0,226.10,Forward,1
4,Juan Martín Fernandez Lobbe,191,106,69,25,5,2,0,202.46,Forward,1


In [66]:
#Calculate Average Points and Average Tries
combined["Avg_Pts"] = combined.Points_Scored.astype("float")/combined.Matches.astype("float")
combined["Avg_Tries"] = combined.Tries_Scored.astype("float")/combined.Matches.astype("float")
combined.head()

,Name,Height,Weight,Matches,Points_Scored,Tries_Scored,Yellow_Cards,Red_Cards,Mass,Field_Pos,Target,Avg_Pts,Avg_Tries
0,Matias Alemanno,198,112,18,10,2,0,0,221.76,Forward,1,0.555556,0.111111
1,Marcos Ayerza,186,113,64,5,1,1,0,210.18,Forward,1,0.078125,0.015625
2,Agustín Creevy,182,106,45,10,2,0,0,192.92,Forward,1,0.222222,0.044444
3,Juan Pablo Orlandi,190,119,19,0,0,0,0,226.10,Forward,1,0.000000,0.000000
4,Juan Martín Fernandez Lobbe,191,106,69,25,5,2,0,202.46,Forward,1,0.362319,0.072464


In [68]:
#For players who have played no games, fill Nan values with 0
combined.Avg_Pts = combined.Avg_Pts.fillna(0)
combined.Avg_Tries = combined.Avg_Tries.fillna(0)

Name             Rory Sutherland
Height                       180
Weight                       110
Matches                        0
Points_Scored                  0
Tries_Scored                   0
Yellow_Cards                   0
Red_Cards                      0
Mass                         198
Field_Pos                Forward
Target                         1
Avg_Pts                        0
Avg_Tries                      0
Name: 180, dtype: object

In [69]:
combined=combined.drop(["Field_Pos", "Matches", "Points_Scored", "Tries_Scored"], axis = 1)
combined.head()

,Name,Height,Weight,Yellow_Cards,Red_Cards,Mass,Target,Avg_Pts,Avg_Tries
0,Matias Alemanno,198,112,0,0,221.76,1,0.555556,0.111111
1,Marcos Ayerza,186,113,1,0,210.18,1,0.078125,0.015625
2,Agustín Creevy,182,106,0,0,192.92,1,0.222222,0.044444
3,Juan Pablo Orlandi,190,119,0,0,226.10,1,0.000000,0.000000
4,Juan Martín Fernandez Lobbe,191,106,2,0,202.46,1,0.362319,0.072464


In [70]:
#Normalise and Scale values
to_norm = ["Height", "Weight", "Mass", "Avg_Pts", "Avg_Tries"]
for factor in to_norm:
    factor_norm = factor + "_norm"
    factor_scale = factor + "_scaled"
    combined[factor_norm] = (combined[factor] - combined[factor].mean()) / (combined[factor].max() - combined[factor].min())
    combined[factor_scale] = (combined[factor] - combined[factor].min()) / combined[factor].max()
    
combined.head()

,Name,Height,Weight,Yellow_Cards,Red_Cards,Mass,Target,Avg_Pts,Avg_Tries,Height_norm,Height_scaled,Weight_norm,Weight_scaled,Mass_norm,Mass_scaled,Avg_Pts_norm,Avg_Pts_scaled,Avg_Tries_norm,Avg_Tries_scaled
0,Matias Alemanno,198,112,0,0,221.76,1,0.555556,0.111111,0.259356,0.149038,0.103206,0.289655,0.153056,0.367093,-0.044742,0.038949,-0.023894,0.095238
1,Marcos Ayerza,186,113,1,0,210.18,1,0.078125,0.015625,-0.033327,0.091346,0.116539,0.296552,0.084434,0.326553,-0.078214,0.005477,-0.105739,0.013393
2,Agustín Creevy,182,106,0,0,192.92,1,0.222222,0.044444,-0.130888,0.072115,0.023206,0.248276,-0.017847,0.266130,-0.068112,0.015580,-0.081037,0.038095
3,Juan Pablo Orlandi,190,119,0,0,226.10,1,0.000000,0.000000,0.064234,0.110577,0.196539,0.337931,0.178775,0.382286,-0.083691,0.000000,-0.119132,0.000000
4,Juan Martín Fernandez Lobbe,191,106,2,0,202.46,1,0.362319,0.072464,0.088624,0.115385,0.023206,0.248276,0.038686,0.299527,-0.058290,0.025402,-0.057020,0.062112


In [71]:
target = combined.Target
combined = combined.drop(["Name", "Target"], axis = 1)
combined.head()

,Height,Weight,Yellow_Cards,Red_Cards,Mass,Avg_Pts,Avg_Tries,Height_norm,Height_scaled,Weight_norm,Weight_scaled,Mass_norm,Mass_scaled,Avg_Pts_norm,Avg_Pts_scaled,Avg_Tries_norm,Avg_Tries_scaled
0,198,112,0,0,221.76,0.555556,0.111111,0.259356,0.149038,0.103206,0.289655,0.153056,0.367093,-0.044742,0.038949,-0.023894,0.095238
1,186,113,1,0,210.18,0.078125,0.015625,-0.033327,0.091346,0.116539,0.296552,0.084434,0.326553,-0.078214,0.005477,-0.105739,0.013393
2,182,106,0,0,192.92,0.222222,0.044444,-0.130888,0.072115,0.023206,0.248276,-0.017847,0.266130,-0.068112,0.015580,-0.081037,0.038095
3,190,119,0,0,226.10,0.000000,0.000000,0.064234,0.110577,0.196539,0.337931,0.178775,0.382286,-0.083691,0.000000,-0.119132,0.000000
4,191,106,2,0,202.46,0.362319,0.072464,0.088624,0.115385,0.023206,0.248276,0.038686,0.299527,-0.058290,0.025402,-0.057020,0.062112


In [72]:
X = combined.values
Y = target.values

In [73]:
from sklearn.cross_validation import cross_val_score

In [96]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
#lr.fit(X_train, y_train)
#lr.score(X_test, y_test)
scores = cross_val_score(lr, X, Y, cv=5)
print scores.mean()
print scores.std()

0.92358490566
0.0299390637607


In [104]:
from sklearn.ensemble import RandomForestClassifier

n_list = [10, 100, 1000]
scores_dict = {}
for n in n_list:
    rfc = RandomForestClassifier(n_estimators=n)
    scores = cross_val_score(rfc, X, Y, cv = 5)
    scores_dict[n] = [scores.mean(), scores.std()]

print scores_dict

{1000: [0.91589259796806977, 0.029068729898390645], 10: [0.91204644412191593, 0.026475830905884977], 100: [0.91966618287373003, 0.031098648748480867]}


In [109]:
rfc.fit(X,Y)
zip(combined.columns, rfc.feature_importances_)

[('Height', 0.015165671758906363),
 ('Weight', 0.14824244471826797),
 ('Yellow_Cards', 0.0055677995488369681),
 ('Red_Cards', 0.0001708827187224071),
 ('Mass', 0.07881966368101305),
 ('Avg_Pts', 0.046478486955101796),
 ('Avg_Tries', 0.025321161814817535),
 ('Height_norm', 0.016051514162409325),
 ('Height_scaled', 0.015128701204641855),
 ('Weight_norm', 0.15377791494987858),
 ('Weight_scaled', 0.17743215464655693),
 ('Mass_norm', 0.082821802054216798),
 ('Mass_scaled', 0.091573316664053855),
 ('Avg_Pts_norm', 0.045927152029492962),
 ('Avg_Pts_scaled', 0.048191337568940097),
 ('Avg_Tries_norm', 0.025083633150112524),
 ('Avg_Tries_scaled', 0.024246362374030991)]

In [103]:
from sklearn.neighbors import KNeighborsClassifier

scores_dict = {}

for n in range(1,15,2):
    knn = KNeighborsClassifier(n_neighbors = n)
    scores = cross_val_score(knn, X, Y, cv = 10)
    scores_dict[n] = [scores.mean(), scores.std()]

print scores_dict    

{1: [0.88154415954415966, 0.060086384353112768], 3: [0.89960113960113974, 0.051519274050951268], 5: [0.89960113960113974, 0.054314763986440202], 7: [0.90358974358974375, 0.042187035255165616], 9: [0.88803988603988615, 0.064038975488147673], 11: [0.88803988603988615, 0.064038975488147673], 13: [0.88033618233618238, 0.061664408680334466]}
